In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from dataclasses import dataclass
from typing import Optional, List
from fuzzywuzzy import fuzz
import evaluate

/u/amo-d1/grad/sma340/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/amo-d1/grad/sma340/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [5]:
conversation_training_path = "data/dataset_v2/training"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_training_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_training_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r', encoding='UTF-8') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_train = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_train)}")

100%|██████████| 8720/8720 [00:22<00:00, 387.40it/s]

8720
8720
8720
8720

number of rows: 8720


In [6]:
df_recommender_train = df_recommender_train[(df_recommender_train["recommend_indexes"] != -1) & (df_recommender_train["turns"].apply(lambda x: len(x) > 0))]

In [7]:
conversation_validation_path = "data/dataset_v2/validation"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_validation_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_validation_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r', encoding='UTF-8') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_validation = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_validation)}")

100%|██████████| 1285/1285 [00:03<00:00, 336.80it/s]

1285
1285
1285
1285

number of rows: 1285


In [8]:
df_recommender_validation = df_recommender_validation[(df_recommender_validation["recommend_indexes"] != -1) & (df_recommender_validation["turns"].apply(lambda x: len(x) > 0))]

In [9]:
count = 0
for turn in df_recommender_train['turns']:
    if len(turn) == 0:
        count += 1
print(count)

count = 0
for recommend_index in df_recommender_train['recommend_indexes']:
    if recommend_index == -1:
        count += 1
print(count)

0
0


In [10]:
count = 0
for turn in df_recommender_validation['turns']:
    if len(turn) == 0:
        count += 1
print(count)

count = 0
for recommend_index in df_recommender_validation['recommend_indexes']:
    if recommend_index == -1:
        count += 1
print(count)

0
0


In [11]:
df_recommender_validation

,user_id,previous_interactions,recommended_app_name,turns,recommend_indexes
0,vtg1e7gwd7akgogt,"drag battle, twitch: live game streaming, mam...",uno!ãâ¯ãâ¿ãâ½,"[computer: hello! what brings you here today?,...",22
1,kp38ptk4vwffenzn,"blockstarplanet, lotsa slots - casino games",snapchat,[computer: hi there! what are you looking for ...,20
2,4704gqsclz6315op,"golf clash, xender - share music transfer, nf...",video downloader,[computer: hi there! how can i assist you toda...,2
3,rnvsafiijygjzyvk,"csr classics, tata cliq shopping app india, f...",piano fire: edm music & piano,"[computer: hello! how can i help you today?, h...",22
4,tmxwkzm2cat22pvh,"the house of da vinci, pet rescue saga, facem...",facebook,[computer: hi there! how can i help you today?...,22
...,...,...,...,...,...
1280,rsbsgtmy4ncjvhdx,"waze - gps, maps, traffic alerts & live navig...",bitmoji,[computer: hey there! what are you looking for...,12
1281,nzhuiko02pyj8vro,video editor app - vivacut,music player - mp3 player,[computer: hi there! how can i help you today?...,24
1282,ldzrids0o452udvr,"the frostrune, ark: survival evolved, street ...",colornote notepad notes,[computer: hi there! how can i help you today?...,20
1283,lekzvahdmv1bcian,"pandora - music & podcasts, kingdom rush veng...",minecraft,"[computer: hi! how can i help you today?, huma...",20


In [12]:
df_recommender_train

,user_id,previous_interactions,recommended_app_name,turns,recommend_indexes
0,mw3eetfcrkivh692,"calm - meditate, sleep, relax, kahoot! play &...",fishdom,"[computer: hi! how can i help you today?, huma...",20
1,7ttbcevntpi81xyq,None,pokãâ¯ãâ¿ãâ½mon quest,[computer: hi! need help finding a pokãâãâ©...,14
2,xj3hkxd3ms7sbgrk,"cat escape, math kids: math games for kids, r...",sonic dash - endless running,[computer: hi there! how can i help you today?...,24
3,pdfiesy5vzbpkz58,"animal jam, brave private web browser, starz,...",pixel gun 3d - battle royale,[computer: hello there! how can i assist you t...,20
4,p1putumtxe7yw3vl,"war heroes: strategy card game, meetme: chat ...",stick war: legacy,"[computer: hello! how can i help you today?, h...",20
...,...,...,...,...,...
8715,dxkdnbbfqx25oqa9,"google go: a lighter, faster way to search, a...",whatsapp messenger,[computer: hi there! how can i help you today?...,18
8716,r0ad7rnsodtnrkea,"speedtest by ookla, kitten match, gamee prize...",pk xd - play with your friends,[computer: hello there! how can i help you tod...,20
8717,1c30afayr0kyblrc,"stickman party: 1 2 3 4 player games free, li...",ludo club - fun dice game,[computer: hi there! how can i help you today?...,18
8718,mkh0usitaxezqehu,"warhammer 40,000: lost crusade, hot wheels un...",garena free fire max,[computer: hi there! how can i assist you toda...,18


In [13]:
model_checkpoint = "gpt2"
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint, bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["COMPUTER:", "HUMAN:"])
tokenizer.padding_side='left'
tokenizer.truncation_side='left'

model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
model.resize_token_embeddings(len(tokenizer))
model_max_length=1024

In [14]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [15]:
items_train = []
for _, row in df_recommender_train.iterrows():
    prompt = bos
    for index, turn in enumerate(row["turns"]):
        prompt += turn + "\n"
        if (index + 1) < len(row["turns"]) and index % 2 == 1: # only add the training sample where the computer is speaking
            items_train.append(RecommenderItem(prompt, row["turns"][index + 1] + eos))

In [16]:
items_train[12]

RecommenderItem(prompt="<|startoftext|>computer: hi! need help finding a pokã\x83â\x83ã\x82â©mon game app for android?\nhuman: yes, looking for one with fun battle features.\ncomputer: okay with apps that have ads and in-app purchases?\nhuman: yes, that's fine.\n", generation='computer: how about apps with at least 1m user reviews?<|endoftext|>')

In [17]:
items_validation = []
for _, row in df_recommender_validation.iterrows():
    prompt = bos
    for index, turn in enumerate(row["turns"]):
        prompt += turn + "\n"
        if (index + 1) < len(row["turns"]) and index % 2 == 1: # only add the training sample where the computer is speaking
            items_validation.append(RecommenderItem(prompt, row["turns"][index + 1] + eos))

In [18]:
items_train[70].prompt

"<|startoftext|>computer: hey there! how can i help you today?\nhuman: i want to find a vpn proxy app for my android phone.\ncomputer: are you looking for a fast connection in the app?\nhuman: yes, i want an app with a fast connection.\ncomputer: do you want the app to have high ratings, like 4.7 stars or above?\nhuman: yes, i prefer apps with high ratings.\ncomputer: do you mind if the app has some ads?\nhuman: i don't mind a few ads.\ncomputer: how important is the number of reviews for the app?\nhuman: i prefer apps with a significant number of reviews, like 1m.\ncomputer: are you looking for a free app or are you open to paying for additional features?\nhuman: i prefer free apps.\ncomputer: based on your preferences, i recommend trying out hotspotshield vpn & wifi proxy app.\nhuman: can you tell me more about the ads in this app?\ncomputer: users have mentioned that there are overwhelming ads.\nhuman: i'd rather not deal with overwhelming ads. do you have any other suggestions?\nco

In [19]:
items_train[0].prompt

"<|startoftext|>computer: hi! how can i help you today?\nhuman: i'm looking for puzzle game apps for android devices.\n"

In [20]:
items_train[0].generation

'computer: what type of puzzle games do you like?<|endoftext|>'

In [21]:
def training_collator(batch: list[recommenderDataset]): # for training a language model
    input_ids = []
    attention_masks = []
    labels = []
    for item in batch:
        prompt_tokens = tokenizer.encode(item.prompt, return_tensors="pt")[0] 
        generation_tokens = tokenizer.encode(item.generation, return_tensors="pt")[0]
        prompt_len = len(prompt_tokens)
        generation_len = len(generation_tokens)
        unused_len = model_max_length - prompt_len - generation_len
        # handling case when input is greater than tokenizer length.
        if unused_len < 0:
            prompt_start_tokens = prompt_tokens[:1]
            trimmed_prompt = prompt_tokens[unused_len * -1 + 1 :] # TODO: you could delete the prompt to reach the first |beginuser| token
            prompt_tokens = torch.cat(
                [prompt_start_tokens, trimmed_prompt], axis=0
            )
            prompt_len = len(prompt_tokens)
            unused_len = 0
        pad = torch.full([unused_len], tokenizer.pad_token_id)
        input_tokens = torch.cat(
            [prompt_tokens, generation_tokens, pad]
        )
        label = torch.cat(
            [
                torch.full(
                    [prompt_len],
                    -100,
                ),
                generation_tokens,
                torch.full([unused_len], -100),
            ]
        )
        attention_mask = torch.cat(
            [
                torch.full([prompt_len + generation_len], 1),
                torch.full([unused_len], 0),
            ]
        )
        input_ids.append(input_tokens)
        attention_masks.append(attention_mask)
        labels.append(label)

    out = {
        "input_ids": torch.stack(input_ids),
        "attention_mask": torch.stack(attention_masks),
        "labels": torch.stack(labels),
    }

    return out

In [22]:
training_args = TrainingArguments(
    output_dir="models/GPT2_responder",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=4500,#self.cfg.save_steps,
    eval_steps=4500, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)


trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=recommenderDataset(items_train),
        eval_dataset=recommenderDataset(items_validation), #dm.datasets[DataNames.dev_language_model.value],
        data_collator=training_collator,
    )

In [23]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
4500,0.856500,0.791229
9000,0.660200,0.666320
13500,0.608500,0.605943
18000,0.547600,0.579657
22500,0.537200,0.558049
27000,0.501100,0.548798
31500,0.473200,0.542862
36000,0.464300,0.536625


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [24]:
torch.cuda.empty_cache()

# Load the model and test it on the test dataset

In [25]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [26]:
conversation_test_path = "data/dataset_v2/testing"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_test_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_test_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_test = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_test)}")

100%|██████████| 2557/2557 [00:05<00:00, 450.17it/s]

2557
2557
2557
2557

number of rows: 2557


In [27]:
df_recommender_test = df_recommender_test[(df_recommender_test["recommend_indexes"] != -1) & (df_recommender_test["turns"].apply(lambda x: len(x) > 0))]

In [28]:
@dataclass
class RecommenderItem:
    prompt: str
    generation: Optional[str] = None
    
class recommenderDataset(Dataset):
    def __init__(self, data: List[RecommenderItem]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx) -> RecommenderItem:
        return self.data[idx]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bos = '<|startoftext|>'
eos = '<|endoftext|>'
pad = '<|pad|>'

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token=bos, eos_token=eos, pad_token=pad, additional_special_tokens=["computer:", "human:"])
tokenizer.padding_side='left'
tokenizer.truncation_side='left'

model = GPT2LMHeadModel.from_pretrained("models/GPT2_responder").to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [31]:
items_test = []
for _, row in df_recommender_test.iterrows():
    prompt = bos
    for index, turn in enumerate(row["turns"]):
        prompt += turn + "\n"
        if (index + 1) < len(row["turns"]) and index % 2 == 1: # only add the training sample where the computer is speaking
            items_test.append(RecommenderItem(prompt, row["turns"][index + 1] + eos))

In [32]:
for item in items_test:
    print(item.generation)

computer: what kind of strategy game apps are you interested in?<|endoftext|>
computer: do you prefer apps with many reviews?<|endoftext|>
computer: are you okay with apps that have ads and offer in-app purchases?<|endoftext|>
computer: do you prefer free apps or are you open to paying for premium features?<|endoftext|>
computer: do you like apps with a high average rating from users?<|endoftext|>
computer: based on what you've told me, i suggest trying rise of the kings app.<|endoftext|>
computer: it has great graphics and fun gameplay, but there are reports of expensive in-game purchases.<|endoftext|>
computer: how about trying state of survival: zombie war?<|endoftext|>
computer: it offers cooperative gameplay and various events, but there are concerns about pay-to-win.<|endoftext|>
computer: in that case, i recommend trying stick war: legacy app.<|endoftext|>
computer: stick war: legacy offers high-quality graphics and regular updates.<|endoftext|>
computer: are you interested in s

In [33]:
bleu = evaluate.load("bleu")

In [34]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_responder(dataset, model, tokenizer, batch_size=8, device=device, bleu=bleu):
  prompt_arr = [data.prompt for data in dataset]
  generation_arr = [data.generation for data in dataset]
  prompt_batches = list(chunk(prompt_arr, batch_size))
  generation_batches = list(chunk(generation_arr, batch_size))
  prompt_size = 896
  generation_size = 128
  predictions = []
  references = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=prompt_size, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device),
                            max_new_tokens=generation_size,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    generations_predicted = generations_predicted[:, prompt_size:] # we only need the generation part, not the prompt part.
    decoded_generations = [tokenizer.decode(generation, clean_up_tokenization_spaces=True).replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")   for generation in generations_predicted]
    generation_batch = [generation.replace(tokenizer.eos_token, "") for generation in generation_batch]
    
    predictions.extend(decoded_generations)
    references.extend([[gen] for gen in generation_batch])

  results = bleu.compute(predictions=predictions, references=references)
  
  return results

In [35]:
results = evaluate_responder(recommenderDataset(items_test), model, tokenizer, batch_size=4, device=device, bleu=bleu)
print("results: ", results)

100%|██████████| 6899/6899 [3:52:51<00:00,  2.03s/it]  


results:  {'bleu': 0.19341461692225442, 'precisions': [0.351906655494258, 0.20227625989635908, 0.1564567501117568, 0.12565809379727685], 'brevity_penalty': 1.0, 'length_ratio': 1.0479322902787824, 'translation_length': 412922, 'reference_length': 394035}
